# **1. Problem Definition**

**Problem Statement:**
To develop a machine learning–based regression model that predicts the compressive strength of concrete using mix design parameters and curing age.

**Input variables (features):**
Cement, Slag, Fly Ash, Water, Plasticizer, Coarse Aggregate, Fine Aggregate, and Curing Age

**Output variable (target):**
Compressive Strength of concrete (MPa)

**Type of problem:**
Supervised machine learning – Regression

**Objective:**
Learn the relationship between concrete mix composition and curing age with compressive strength, enabling accurate strength prediction without conducting physical compressive tests for every mix.

# **2. Import required librarires**

**Pandas:** Reads and organizes concrete data
**NumPy:**	Performs numerical calculations
**Matplotlib:**	Basic plotting and visualization
**Seaborn:**	Advanced statistical visualization

Together, they form the data analysis and visualization layer before applying ML models.

In [ ]:
import pandas as pd
#Pandas is a Python library used for data handling and manipulation. Works similar to excel

import numpy as np
#NumPy is a library for numerical and mathematical operations (matrix algebra)

import matplotlib.pyplot as plt
#Matplotlib is a plotting library used to create graphs (visualization)

import seaborn as sns
#Seaborn is built on top of Matplotlib that provides statistical visualziations

# **3. Data Collection / Loading**

In [ ]:
#Read the csv flie using pandas

#CSV (Comma Separated Values) is a common format for experimental data similar to excel sheets
#Each row → one concrete mix design; Each column → one parameter (cement, water, age, strength, etc.)

data = pd.read_csv('/content/Compressive_Strength.csv')
#Reads a CSV file and converts it into a DataFrame. The DataFrame is a tabular data structure with rows and columns
#This is equivalent to opening an Excel file and loading it into memory for analysis.

df = data.copy()
#creates an independent duplicate of the data

# **4. Exploratory Data Analysis (EDA)**

In [ ]:
df.head()
#Displays the first 5 rows of the dataset by default
#necessary to confirm whether the data loaded correctly, columns are in expected order, and no obvious errors

In [ ]:
df.tail()

In [ ]:
df.info()
'''
Provides a summary of the dataset
Displays number of rows and columns; column names; data type of each column;
Count of non-null (non-missing) values; #Memory usage

It is necessary to check whether the columns are numerical or categorical, are there any missing values,
and completeness of data.
'''


In [ ]:
df.shape
#Shows the number of rows and columns present in the dataset
#Necessary to know the size of the dataset before proceeding with analysis


In [ ]:
df.describe()

#Generates a statistical summary of all numerical columns in the dataset
#Necessary to understand the data spread, typical values, and potential outliers

In [ ]:
df.isnull().sum()
#Checking missing values

**Visulalization**

In [ ]:
plt.figure(figsize=(12, 6))
#creates a new figure and sets the width and height

sns.boxplot(data=df)
#Uses Seaborn to draw a box plot for each column in the DataFrame df
#Automatically:Computes quartiles (Q1, median, Q3); Identifies whiskers and outliers
#X-axis → Variables (Cement, Water, Age, Strength, etc.)
#Y-axis → Numerical values

plt.xticks(rotation=90)
#Rotates x-axis labels by 90 degrees (vertical)

plt.title("Box Plots of All Features")
#Adds a title to the plot

plt.show()
#Dislplays the plot on the screen

What does a box plot show?
*   Box → Middle 50% of data (Interquartile Range, IQR)
*   Line inside box → Median
*   Whiskers → Acceptable data range
*   Dots outside whiskers → Outliers

Outliers are the data points that lie significantly outside the normal range of observations which could be due to experimental errors and should be examined before removing

In [ ]:
#Numerical depiction of the box plot

'''
This code:
Numerically identifies outliers for every numerical variable in the dataset
Uses the Interquartile Range (IQR) method (same logic as box plots)
Produces a summary table showing how many outliers each variable has
'''

outlier_summary = []
#creates an empty list

for col in df.select_dtypes(include=["int64", "float64"]).columns: #loops through numerical columns only
    Q1 = df[col].quantile(0.25) #computes frist quartile (25% data lies below this value)
    Q3 = df[col].quantile(0.75) #computes third quartile (75% data lies below this value)
    IQR = Q3 - Q1 #computes interquartile range (represents middle 50% of the data)

    lower_bound = Q1 - 1.5 * IQR #Calculates the lower acceptable limit
    upper_bound = Q3 + 1.5 * IQR #Calculates the upper acceptable limit

    outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
    #filters rows where values are either less than lower bound or greater than upper bound
    #.shape[0] counts how many such rows exist

    outlier_summary.append({
        "Variable": col,
        "Q1": Q1,
        "Q3": Q3,
        "IQR": IQR,
        "Lower Bound": lower_bound,
        "Upper Bound": upper_bound,
        "Number of Outliers": outlier_count
    })
    #Appends a dictionary of results for the current variable
    #Each dictionary represents one row in the final table

outlier_table = pd.DataFrame(outlier_summary) #converts the list of dictionaries into a pandas dataframe
outlier_table #displays the final outlier summary table


In [ ]:
#generate pair plot

sns.pairplot(df)
#Creates pairwise plots between all the numerical variables in the dataset
#Necessary to examine relationships between variables
#To understand how each input parameter relates to the output (strength)

**Strength vs Age → Strong, Clearly Non-Linear Relationship**


Strength increases with age

Dense vertical bands at specific ages (1, 3, 7, 14, 28, 56, 90, 365 days)

At low ages → wide spread, lower strength

At high ages → higher strength but plateauing

**Interpretation**

Strength gain follows hydration kinetics

Early-age strength gain is rapid

Long-term gain slows → non-linear saturation behavior

**Key learning point**

This relationship is not linear; it is time-dependent and asymptotic.

**ML implication**

Linear regression will underpredict long-term strength

Tree-based models and ANN are justified

In [ ]:
#Making non linerity explicit

df.groupby("Age")["Strength "].mean().plot(marker='o')
#groups the dataset by curing age. All samples tested at the same age are collected together

plt.xlabel("Age")
plt.ylabel("Mean Strength")
plt.title("Mean Strength vs Age")
plt.show()

In [ ]:
#generating heatmap

#1: compute the correlation matrix

corr = df.corr()
#Computes the Pearson correlation coefficient between every pair of numerical variables
#correlation measures linear relationship between two variables (linear dependence only)
#(+1) - perfect positive linear correlation
#(0) - no liner correlation
#(-1) - perfect negative linear correlation

#2: plot the heatmap
#sns.heatmap(corr, annot=True)

mask = np.triu(np.ones_like(corr, dtype=bool))
#create a mask for upper triangle
#np.ones_like(corr) - Creates a matrix of ones with the same shape as corr
#dtype=bool - Converts the matrix to True/False values
#np.triu(...) - Keeps only the upper triangular part, Lower triangle becomes False

sns.heatmap(corr, mask = mask, annot=True, cmap="Blues", fmt ='.2f')
#corr - Data being visualized (correlation matrix)
#mask=mask - Hides the upper triangle
#annot=True - Prints numerical correlation values in each cell
#cmap="Blues" - Color map selection
#fmt='.2f' - Formats numbers to 2 decimal places

**Most Important Row: Strength (Last Row)**

**Cement → Strength (+0.50)**

🔹Strongest positive correlation in the dataset

🔹Indicates cement content is a primary strength driver

**Plasticizer → Strength (+0.37)**

🔹Moderate positive correlation

🔹Indicates indirect influence via Improved workability and Reduced water demand

**Age → Strength (+0.33)**

🔹Moderate positive correlation

🔹Confirms strength increases with curing time

**Water → Strength (–0.29)**

🔹Clear negative correlation

🔹Matches classic water–strength relationship

---
---

**Important Relationships Between Input Variables**

**Water vs Plasticizer (–0.66)**

🔹Strongest correlation in the entire heatmap

🔹Very important result

**Fine Aggregate vs Water (–0.45)**

🔹Moderate negative correlation

🔹Denser fine aggregate packing reduces water demand

**Cement vs Fly Ash (–0.40)**

🔹Strong negative correlation

🔹Indicates cement replacement by fly ash

**Cement vs Slag (–0.28)**

🔹Similar replacement behavior

🔹This confirms mix design trade-offs, not data issues.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Compute correlation
corr = df.corr()

# Create mask for upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr,
    mask=mask,
    annot=True,
    cmap="Blues",
    fmt=".2f",
    cbar=True
)

# ---- ADD INTERPRETATION ANNOTATIONS ---- #

# Cement – Strength
plt.text(0.5, 8.5, "Strong\npositive\ninfluence",
         color="yellow", ha="left", va="center", fontsize=9)

# Water – Strength
plt.text(3.5, 8.5, "Inverse\nwater–strength\nrelation",
         color="yellow", ha="left", va="center", fontsize=9)

# Age – Strength
plt.text(7.5, 8.5, "Age effect is\nnon-linear\n(saturation)",
         color="black", ha="left", va="center", fontsize=9)

# Plasticizer – Water
plt.text(4.5, 3.5, "Plasticizer\nreduces\nwater demand",
         color="black", ha="center", va="center", fontsize=9)

# Cement – Fly Ash
plt.text(2.5, 0.5, "Binder\nreplacement\ntrend",
         color="black", ha="center", va="center", fontsize=9)

# Title
plt.title("Correlation Heatmap with Engineering Interpretation", fontsize=12)

plt.tight_layout()
plt.show()


# **4. Data Cleaning**

Handle missing values

Remove or correct:

🔹Invalid entries

🔹Outliers (if physically unjustified)

🔹Ensure consistent units

None of these are required for this dataset

# **5. Feature Engineering**

Create or modify input variables to improve learning

Examples

🔹Water–cement ratio

🔹Total binder content

🔹Age categories (early / long-term curing)

Feature engineering helps ML models capture known concrete behavior more explicitly, especially non-linear and interaction effects.

In [ ]:
df["Water_Cement_Ratio"] = df["Water"] / df["Cement"]
#Feature 1: Water–Cement Ratio (Most Important); Engineering basis: Abram’s law

df["Total_Binder"] = df["Cement"] + df["Slag"] + df["Fly_Ash"]
#Feature 2: total binder content including supplementary cementitous materials (SCMs)

df["SCM_Replacement_Ratio"] = (df["Slag"] + df["Fly_Ash"]) / df["Total_Binder"]
#Feature 3: SCM Replacement Ratio; Fraction of binder replaced by SCMs

df["Water_Binder_Ratio"] = df["Water"] / df["Total_Binder"]
#Feature 4: Water–Binder Ratio; More general than w/c for blended concretes

df["Aggregate_Binder_Ratio"] = (
    df["Coarse_Aggregate"] + df["Fine_Aggregate"]
) / df["Total_Binder"]
#Feature 5: Aggregate–Binder Ratio; Reflects packing density influence

df["Log_Age"] = np.log(df["Age"])
#Feature 6: Age Transformation (to capture non-linearity); Hydration kinetics are logarithmic-like

df["SCM_Age_Interaction"] = df["SCM_Replacement_Ratio"] * df["Log_Age"]
#Optional: Interaction Features; Age × SCM interaction (long-term strength contribution)

In [ ]:
df.head()

In [ ]:
df.columns

# **8. Feature - Lable (target) separation**

In [ ]:
#extracting features and label

X = df.drop("Strength ", axis=1)
#Creates a new DataFrame X; Contains all columns except "Strength"
#drop() removes a column or row
#"Strength" → column to be removed
#axis=1 → specifies column-wise removal
#capital letter indicates a matrix (feature matrix)

y = df["Strength "]
#Extracts the Strength column only; Stores it as a separate variable y (target vector)

In [ ]:
X

In [ ]:
y

# **9. Train - Test Split**

split the dataset into training and testing subsets, so that:

🔹The model learns from one portion of data

🔹The model is evaluated on unseen data

This is a mandatory step in any machine-learning workflow.

In [ ]:
#splitting data into train and test sets

from sklearn.model_selection import train_test_split
#Imports the train_test_split function from scikit-learn
#This function is used to randomly divide data into training and test sets
#scikit-learn is the standard ML library in Python

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X,y: Features and target are passed separately
#test_size=0.2: Specifies that 20% of the data is used for testing; Remaining 80% is used for training
#random_state=42: Fixes the random number generator; Ensures same split every time the code is run and reproducibility of results

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    print(f"\n{model_name} Performance:")
    print(f"R²   : {r2:.3f}")
    print(f"MAE  : {mae:.3f}")
    print(f"MSE  : {mse:.3f}")
    print(f"RMSE : {rmse:.3f}")

    return r2, mae, mse, rmse

# **10. Model training with scaling and cross validation followed by evaluation**

Train–test split is sufficient when:

🔹Dataset is reasonably large (≈ 800–1000+ samples)

🔹Goal is quick model development or teaching

🔹You are not tuning many hyperparameters

🔹Model performance is not highly sensitive to data split

Scaling REQUIRED for (linear models):

🔹Linear Regression

🔹Support Vector Regression (SVR)

🔹Artificial Neural Networks (ANN)

🔹KNN

Scaling NOT required for (non-linear models):

🔹Decision Tree

🔹Random Forest

🔹Gradient Boosting

Cross-validation is required when:

🔹Dataset is moderate in size (≤ 2000 samples)

🔹Data has high variability

🔹You are comparing multiple models

🔹You are tuning hyperparameters

🔹Results are meant for research or publication

In [ ]:
#importing the necessary libraries

from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

In [ ]:
#Define K-Fold cross validation strategy

kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
#Model 1: Linear Regression (Scaling REQUIRED)

# Define model ONCE
lr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

# 1. Cross-validation (model selection)
lr_scores = cross_val_score(
    lr_pipeline, X_train, y_train, cv=kf, scoring="r2"
)

print("Linear Regression (Baseline) CV R²:", lr_scores.mean())

# 2. Final evaluation (test set)
evaluate_model(
    lr_pipeline,
    X_train, X_test,
    y_train, y_test,
    model_name="Linear Regression (Baseline)"
)

In [ ]:
#Model 2: Support Vector Regression (Scaling MANDATORY)

svr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVR(kernel="rbf"))
])

# 1. Cross-validation
svr_scores = cross_val_score(
    svr_pipeline, X_train, y_train, cv=kf, scoring="r2"
)

print("SVR (Baseline) CV R²:", svr_scores.mean())

# 2. Final evaluation
evaluate_model(
    svr_pipeline,
    X_train, X_test,
    y_train, y_test,
    model_name="SVR (Baseline)"
)

In [ ]:
#Model 3: Artificial Neural Network (ANN) (Scaling MANDATORY)

# Model 3: ANN (Scaling REQUIRED)

ann_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", MLPRegressor(
        hidden_layer_sizes=(50, 50),
        max_iter=1000,
        random_state=42
    ))
])

# 1. Cross-validation
ann_scores = cross_val_score(
    ann_pipeline, X_train, y_train, cv=kf, scoring="r2"
)

print("ANN (Baseline) CV R²:", ann_scores.mean())

# 2. Final evaluation
evaluate_model(
    ann_pipeline,
    X_train, X_test,
    y_train, y_test,
    model_name="ANN (Baseline)"
)

In [ ]:
#Model 4: Decision Tree (Scaling NOT Required)

dt_model = DecisionTreeRegressor(random_state=42)

# 1. Cross-validation
dt_scores = cross_val_score(
    dt_model, X_train, y_train, cv=kf, scoring="r2"
)

print("Decision Tree (Baseline) CV R²:", dt_scores.mean())

# 2. Final evaluation
evaluate_model(
    dt_model,
    X_train, X_test,
    y_train, y_test,
    model_name="Decision Tree (Baseline)"
)

In [ ]:
#Model 5: Random Forest (Scaling NOT Required)

rf_model = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)

# 1. Cross-validation
rf_scores = cross_val_score(
    rf_model, X_train, y_train, cv=kf, scoring="r2"
)

print("Random Forest (Baseline) CV R²:", rf_scores.mean())

# 2. Final evaluation
evaluate_model(
    rf_model,
    X_train, X_test,
    y_train, y_test,
    model_name="Random Forest (Baseline)"
)

In [ ]:
#Model 6: Gradient Boosting (Scaling NOT Required)

gb_model = GradientBoostingRegressor(random_state=42)

# 1. Cross-validation
gb_scores = cross_val_score(
    gb_model, X_train, y_train, cv=kf, scoring="r2"
)

print("Gradient Boosting (Baseline) CV R²:", gb_scores.mean())

# 2. Final evaluation
evaluate_model(
    gb_model,
    X_train, X_test,
    y_train, y_test,
    model_name="Gradient Boosting (Baseline)"
)

In [ ]:
#Model 7: XGBoost Regressor (Scaling NOT required)

xgb_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="reg:squarederror"
)

# 1. Cross-validation
xgb_scores = cross_val_score(
    xgb_model, X_train, y_train, cv=kf, scoring="r2"
)

print("XGBoost (Baseline) CV R²:", xgb_scores.mean())

# 2. Final evaluation
evaluate_model(
    xgb_model,
    X_train, X_test,
    y_train, y_test,
    model_name="XGBoost (Baseline)"
)

In [ ]:
# Compare all models using CV R² (model selection)

cv_results = {
    "Linear Regression": lr_scores.mean(),
    "SVR": svr_scores.mean(),
    "ANN": ann_scores.mean(),
    "Decision Tree": dt_scores.mean(),
    "Random Forest": rf_scores.mean(),
    "Gradient Boosting": gb_scores.mean(),
    "XGBoost": xgb_scores.mean()
}

print("Cross-Validated R² Scores:")
for model, score in cv_results.items():
    print(f"{model:20s}: {score:.3f}")

In [ ]:
results = {}

results["Linear Regression"] = evaluate_model(
    lr_pipeline, X_train, X_test, y_train, y_test, "Linear Regression"
)

results["SVR"] = evaluate_model(
    svr_pipeline, X_train, X_test, y_train, y_test, "SVR"
)

results["ANN"] = evaluate_model(
    ann_pipeline, X_train, X_test, y_train, y_test, "ANN"
)

results["Decision Tree"] = evaluate_model(
    dt_model, X_train, X_test, y_train, y_test, "Decision Tree"
)

results["Random Forest"] = evaluate_model(
    rf_model, X_train, X_test, y_train, y_test, "Random Forest"
)

results["Gradient Boosting"] = evaluate_model(
    gb_model, X_train, X_test, y_train, y_test, "Gradient Boosting"
)

results["XGBoost"] = evaluate_model(
    xgb_model, X_train, X_test, y_train, y_test, "XGBoost"
)

In [ ]:
import pandas as pd

results_df = pd.DataFrame.from_dict(
    results,
    orient="index",
    columns=["R2", "MAE", "MSE", "RMSE"]
)

results_df = results_df.sort_values(by="RMSE")

print(results_df)

In [ ]:
# Optional: BASELINE MODEL COMPARISON (NO TUNING); Define all baseline models once

models = {
    "Linear Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),

    "SVR": Pipeline([
        ("scaler", StandardScaler()),
        ("model", SVR(kernel="rbf"))
    ]),

    "ANN": Pipeline([
        ("scaler", StandardScaler()),
        ("model", MLPRegressor(hidden_layer_sizes=(50,50),
                               max_iter=1000,
                               random_state=42))
    ]),

    "Decision Tree": DecisionTreeRegressor(random_state=42),

    "Random Forest": RandomForestRegressor(
        n_estimators=200, random_state=42
    ),

    "Gradient Boosting": GradientBoostingRegressor(random_state=42),

    "XGBoost": XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        objective="reg:squarederror"
    )
}

In [ ]:
# A. Cross-validated R² (Model Selection)

cv_results = {}

for name, model in models.items():
    scores = cross_val_score(
        model, X_train, y_train, cv=kf, scoring="r2"
    )
    cv_results[name] = scores.mean()

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perce

In [ ]:
# B. Test-Set Evaluation (ONCE per model)

test_results = {}

for name, model in models.items():
    test_results[name] = evaluate_model(
        model, X_train, X_test, y_train, y_test, name
    )


Linear Regression Performance:
R²   : 0.840
MAE  : 5.191
MSE  : 41.167
RMSE : 6.416

SVR Performance:
R²   : 0.809
MAE  : 5.400
MSE  : 49.114
RMSE : 7.008

ANN Performance:
R²   : 0.891
MAE  : 3.523
MSE  : 28.098
RMSE : 5.301

Decision Tree Performance:
R²   : 0.833
MAE  : 4.108
MSE  : 43.036
RMSE : 6.560

Random Forest Performance:
R²   : 0.905
MAE  : 3.389
MSE  : 24.575
RMSE : 4.957

Gradient Boosting Performance:
R²   : 0.910
MAE  : 3.476
MSE  : 23.178
RMSE : 4.814

XGBoost Performance:
R²   : 0.919
MAE  : 3.094
MSE  : 20.800
RMSE : 4.561


In [ ]:
# C. Results Table

import pandas as pd

cv_df = pd.DataFrame.from_dict(
    cv_results, orient="index", columns=["CV_R2"]
).sort_values(by="CV_R2", ascending=False)

test_df = pd.DataFrame.from_dict(
    test_results,
    orient="index",
    columns=["R2", "MAE", "MSE", "RMSE"]
).sort_values(by="RMSE")

print("=== Cross-Validated R² (Model Selection) ===")
print(cv_df)

print("\n=== Test Set Performance (Final Evaluation) ===")
print(test_df)

=== Cross-Validated R² (Model Selection) ===
                      CV_R2
XGBoost            0.926986
ANN                0.920740
Gradient Boosting  0.909266
Random Forest      0.901449
Linear Regression  0.832474
Decision Tree      0.829372
SVR                0.782320

=== Test Set Performance (Final Evaluation) ===
                         R2       MAE        MSE      RMSE
XGBoost            0.919280  3.093780  20.800138  4.560717
Gradient Boosting  0.910050  3.476086  23.178401  4.814395
Random Forest      0.904631  3.389472  24.574860  4.957304
ANN                0.890960  3.523196  28.097574  5.300714
Linear Regression  0.840240  5.190949  41.167207  6.416168
Decision Tree      0.832987  4.107719  43.036011  6.560184
SVR                0.809400  5.400039  49.113905  7.008131


# **11. Hyperparameter Tuning**

In [ ]:
#Model 1: Linear Regression has very limited or no hyperparameters to tune and hence Ridge Regression is considered

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge())
])

ridge_param_grid = {
    "model__alpha": [0.01, 0.1, 1, 10, 100]
}

ridge_grid = GridSearchCV(
    ridge_pipeline,
    ridge_param_grid,
    cv=kf,
    scoring="r2"
)

ridge_grid.fit(X_train, y_train)

print("Best Ridge parameters:", ridge_grid.best_params_)
print("Best CV R²:", ridge_grid.best_score_)

In [ ]:
#Model 2: Support Vector Regression (SVR)

from sklearn.svm import SVR

svr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVR())
])

svr_param_grid = {
    "model__C": [1, 10, 100],
    "model__gamma": ["scale", 0.01, 0.1],
    "model__epsilon": [0.01, 0.1, 0.2]
}

svr_grid = GridSearchCV(
    svr_pipeline,
    svr_param_grid,
    cv=kf,
    scoring="r2",
    n_jobs=-1
)

svr_grid.fit(X_train, y_train)

print("Best SVR parameters:", svr_grid.best_params_)
print("Best CV R²:", svr_grid.best_score_)

In [ ]:
#Model 3: ANN (MLPRegressor) – Use RandomizedSearchCV

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV

ann_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", MLPRegressor(max_iter=1000, random_state=42))
])

ann_param_dist = {
    "model__hidden_layer_sizes": [(50,), (100,), (50,50), (100,50)],
    "model__alpha": [0.0001, 0.001, 0.01],
    "model__learning_rate_init": [0.001, 0.01]
}

ann_search = RandomizedSearchCV(
    ann_pipeline,
    ann_param_dist,
    n_iter=10,
    cv=kf,
    scoring="r2",
    random_state=42,
    n_jobs=-1
)

ann_search.fit(X_train, y_train)

print("Best ANN parameters:", ann_search.best_params_)
print("Best CV R²:", ann_search.best_score_)

In [ ]:
#Model 4: Decision Tree (GridSearchCV)

from sklearn.tree import DecisionTreeRegressor

dt_param_grid = {
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5]
}

dt_grid = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    dt_param_grid,
    cv=kf,
    scoring="r2"
)

dt_grid.fit(X_train, y_train)

print("Best DT parameters:", dt_grid.best_params_)
print("Best CV R²:", dt_grid.best_score_)

In [ ]:
#Model 5: Random Forest (RandomizedSearchCV)

from sklearn.ensemble import RandomForestRegressor

rf_param_dist = {
    "n_estimators": [200, 400, 600],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    rf_param_dist,
    n_iter=20,
    cv=kf,
    scoring="r2",
    n_jobs=-1,
    random_state=42
)

rf_search.fit(X_train, y_train)

print("Best RF parameters:", rf_search.best_params_)
print("Best CV R²:", rf_search.best_score_)

In [ ]:
#Model 6: Gradient Boosting (GridSearchCV)

from sklearn.ensemble import GradientBoostingRegressor

gb_param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 4, 5]
}

gb_grid = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    gb_param_grid,
    cv=kf,
    scoring="r2"
)

gb_grid.fit(X_train, y_train)

print("Best GB parameters:", gb_grid.best_params_)
print("Best CV R²:", gb_grid.best_score_)

In [ ]:
#Model 7: XGBoost (RandomizedSearchCV)

from xgboost import XGBRegressor

xgb_param_dist = {
    "n_estimators": [200, 400, 600],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.03, 0.05, 0.1],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0]
}

xgb_search = RandomizedSearchCV(
    XGBRegressor(
        objective="reg:squarederror",
        random_state=42
    ),
    xgb_param_dist,
    n_iter=20,
    cv=kf,
    scoring="r2",
    n_jobs=-1,
    random_state=42
)

xgb_search.fit(X_train, y_train)

print("Best XGB parameters:", xgb_search.best_params_)
print("Best CV R²:", xgb_search.best_score_)

In [ ]:
best_rf = rf_search.best_estimator_

evaluate_model(
    best_rf,
    X_train, X_test,
    y_train, y_test,
    model_name="Tuned Random Forest"
)

In [ ]:
#Refined Section Hyperparameter Tuning Code (ONLY TOP MODELS)

#Model 1: Tuned XGBoost

from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

xgb_param_dist = {
    "n_estimators": [300, 500, 700],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.03, 0.05, 0.1],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0]
}

xgb_search = RandomizedSearchCV(
    XGBRegressor(
        objective="reg:squarederror",
        random_state=42
    ),
    xgb_param_dist,
    n_iter=25,
    cv=kf,
    scoring="r2",
    n_jobs=-1,
    random_state=42
)

xgb_search.fit(X_train, y_train)

best_xgb = xgb_search.best_estimator_

print("Best XGBoost params:", xgb_search.best_params_)
print("Best XGBoost CV R²:", xgb_search.best_score_)

Best XGBoost params: {'subsample': 0.7, 'n_estimators': 700, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.7}
Best XGBoost CV R²: 0.9291792204392163


In [ ]:
#Model 2: Tuned Gradient Boosting

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

gb_param_grid = {
    "n_estimators": [200, 300, 400],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 4, 5],
    "subsample": [0.8, 1.0]
}

gb_grid = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    gb_param_grid,
    cv=kf,
    scoring="r2",
    n_jobs=-1
)

gb_grid.fit(X_train, y_train)

best_gb = gb_grid.best_estimator_

print("Best GB params:", gb_grid.best_params_)
print("Best GB CV R²:", gb_grid.best_score_)

Best GB params: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 400, 'subsample': 0.8}
Best GB CV R²: 0.9282967372959264


In [ ]:
#Model 3: Tuned Random Forest

from sklearn.ensemble import RandomForestRegressor

rf_param_dist = {
    "n_estimators": [300, 500, 700],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    rf_param_dist,
    n_iter=25,
    cv=kf,
    scoring="r2",
    n_jobs=-1,
    random_state=42
)

rf_search.fit(X_train, y_train)

best_rf = rf_search.best_estimator_

print("Best RF params:", rf_search.best_params_)
print("Best RF CV R²:", rf_search.best_score_)

KeyboardInterrupt: 

In [ ]:
#Final Evaluation of Tuned Models (Test Set)

final_results = {}

final_results["XGBoost (Tuned)"] = evaluate_model(
    best_xgb, X_train, X_test, y_train, y_test
)

final_results["Gradient Boosting (Tuned)"] = evaluate_model(
    best_gb, X_train, X_test, y_train, y_test
)

final_results["Random Forest (Tuned)"] = evaluate_model(
    best_rf, X_train, X_test, y_train, y_test
)

In [ ]:
#Consolidated Final Results Table (Baseline vs Tuned)

final_df = pd.DataFrame.from_dict(
    final_results,
    orient="index",
    columns=["R2", "MAE", "MSE", "RMSE"]
).sort_values(by="RMSE")

print("=== Final Tuned Model Performance ===")
print(final_df)

In [ ]:
# retreive the best model
best_model = grid_search.best_estimator_

# make predictions with the best model
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# generate parity plot for the best model
plt.figure(figsize=(12, 6))

# plot y_train actual vs y_train predicted for the best model
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, color='blue', alpha=0.5)
plt.plot([min(y_train), max(y_train)], [min(y_train), max(y_train)], color='red', linestyle='--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Train Data - Parity Plot')

# plot y_test actual vs y_test predicted for the best model
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred, color='green', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Test Data - Parity Plot')

# display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Feature importance prediction
feature_importances = best_model.feature_importances_

#print the feature importance
print('Feature Importances:')
for feature, importance in zip(X.columns, feature_importances):
  print(f'{feature}: {importance:.4f}')

# plot
plt.figure(figsize=(10, 6))
plt.barh(X.columns, feature_importances)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.xticks(rotation=90)
plt.show